# 1.Data cleaning

## 1.1 Setup

In [1]:
import re
import pandas as pd
from nltk.corpus import stopwords
import nltk
import spacy

## 1.2 Loading Data

In [2]:
# We load our train data into a dataFrame
trainDf = pd.read_excel('resources/referentiel_foodex.xlsx',sheet_name='Feuil1')
trainDf


,Désignation commerciale,Catégorie de référence
0,Lait au chocolat,Chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao)
2,Chocapic en poudre,Poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo..."
4,Jus de pomme bio,Jus de pomme
...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments"
96,Macaron framboise,Pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits


## 1.3 Cleaning steps

### Duplicated rows

In [3]:
# Check if there are duplicated rows 
# No duplicated rows 

duplicates = trainDf[trainDf.duplicated()]
duplicates

,Désignation commerciale,Catégorie de référence


### Missing values

In [4]:
# How much missing values
trainDf.isnull().sum()

Désignation commerciale    0
Catégorie de référence     3
dtype: int64

In [5]:
# There 3 missing categories, for the moment we just remove them 
#-> (any row that has a missing value in a column will be dropped)

trainDf_Cleaned = trainDf.dropna()
trainDf_Cleaned = trainDf_Cleaned.rename(columns={
    'Désignation commerciale':'Product',
    'Catégorie de référence':'Category'})
trainDf_Cleaned

,Product,Category
0,Lait au chocolat,Chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao)
2,Chocapic en poudre,Poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo..."
4,Jus de pomme bio,Jus de pomme
...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments"
96,Macaron framboise,Pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits


### Scientific names handeling

In [6]:
def remove_scientific_names(text):
    # Use regex to find and remove all text between parentheses (scientific names)
    cleaned_text = re.sub(r'\(.*?\)', '', text)
    return cleaned_text

In [7]:
trainDf_Cleaned["Product_clean"] = trainDf_Cleaned["Product"].apply(remove_scientific_names)
trainDf_Cleaned["Category_clean"] = trainDf_Cleaned["Category"].apply(remove_scientific_names)

In [8]:
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,Lait au chocolat,Chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao),Poisson en chocolat,Produits de chocolat
2,Chocapic en poudre,Poudre de cacao,Chocapic en poudre,Poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo..."
4,Jus de pomme bio,Jus de pomme,Jus de pomme bio,Jus de pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",Crème brûlée vanille,"Collations, desserts et autres aliments"
96,Macaron framboise,Pâtisseries et gâteaux,Macaron framboise,Pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits,Tarte tatin,Tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits,Clafoutis aux cerises,Gâteau aux fruits


### Special caracters and Lowercasing

In [9]:
def cleanText(text):
    text = re.sub(r"[^a-zA-Z0-9àâäéèêëîïôöùûüÿçœæÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇŒÆ\s]"," ",text) # Replace special caracters with white space.
    return text.lower() # lowercase

In [10]:
trainDf_Cleaned["Product_clean"] = trainDf_Cleaned["Product_clean"].apply(cleanText)
trainDf_Cleaned["Category_clean"] = trainDf_Cleaned["Category_clean"].apply(cleanText)
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,lait au chocolat,chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao),poisson en chocolat,produits de chocolat
2,Chocapic en poudre,Poudre de cacao,chocapic en poudre,poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",cola light sans bulles,boissons au cola caféiniques faibles en calo...
4,Jus de pomme bio,Jus de pomme,jus de pomme bio,jus de pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",crème brûlée vanille,collations desserts et autres aliments
96,Macaron framboise,Pâtisseries et gâteaux,macaron framboise,pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits,tarte tatin,tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits,clafoutis aux cerises,gâteau aux fruits


### Stopwords removing 

In [11]:
# Download French stopwords if not already
nltk.download('stopwords')
french_stopwords = set(stopwords.words('french'))
len(french_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


157

In [12]:
def remove_french_stopwords(text):
    words = re.findall(r'\b\w+\b', text.lower())
    return ' '.join([w for w in words if w not in french_stopwords])


In [13]:
# Apply to both columns
trainDf_Cleaned['Product_clean'] = trainDf_Cleaned['Product_clean'].apply(remove_french_stopwords)
trainDf_Cleaned['Category_clean'] = trainDf_Cleaned['Category_clean'].apply(remove_french_stopwords)
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,lait chocolat,chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao),poisson chocolat,produits chocolat
2,Chocapic en poudre,Poudre de cacao,chocapic poudre,poudre cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",cola light sans bulles,boissons cola caféiniques faibles calories
4,Jus de pomme bio,Jus de pomme,jus pomme bio,jus pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",crème brûlée vanille,collations desserts autres aliments
96,Macaron framboise,Pâtisseries et gâteaux,macaron framboise,pâtisseries gâteaux
97,Tarte tatin,Tarte aux fruits,tarte tatin,tarte fruits
98,Clafoutis aux cerises,Gâteau aux fruits,clafoutis cerises,gâteau fruits


### Keep only Nouns

use the following command to install the model :

```bash
uv run python -m spacy download fr_dep_news_trf 
```

In [14]:
nlp = spacy.load("fr_dep_news_trf")

In [15]:
def keep_nouns(text):
    doc = nlp(text)
    cleanedText = " ".join([token.text for token in doc if token.pos_ in ["NOUN","PROPN"] ])
    if len(cleanedText) > 0:
        return cleanedText
    return text

In [16]:
# Example 

print(keep_nouns("boissons au cola caféiniques faibles en "))

doc = nlp("boissons au cola caféiniques faibles en ")
for token in doc:
    print(token.text, token.pos_)

boissons cola
boissons NOUN
au ADP
cola NOUN
caféiniques ADJ
faibles ADJ
en ADP


In [17]:
trainDf_Cleaned["Product_clean"] = trainDf_Cleaned["Product_clean"].apply(keep_nouns)
trainDf_Cleaned["Category_clean"] = trainDf_Cleaned["Category_clean"].apply(keep_nouns)

In [18]:
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,lait chocolat,chocolat
1,Poisson en chocolat,Produits de chocolat (cacao),poisson chocolat,produits chocolat
2,Chocapic en poudre,Poudre de cacao,chocapic poudre,poudre cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",cola bulles,boissons calories
4,Jus de pomme bio,Jus de pomme,jus pomme,jus pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",crème vanille,collations desserts aliments
96,Macaron framboise,Pâtisseries et gâteaux,macaron framboise,pâtisseries gâteaux
97,Tarte tatin,Tarte aux fruits,tarte,tarte fruits
98,Clafoutis aux cerises,Gâteau aux fruits,clafoutis cerises,gâteau fruits


### Save the cleaned dataframe

In [19]:
trainDf_Cleaned.iloc[:, -2:].to_excel('data/train_cleaned.xlsx',index=False)